# CMORising ACCESS-ESM1.6 Spin-Up for ACCESS-ENSO-Recipes

This notebook is a cope from a tutorial in [ACCESS-MOPPeR](https://github.com/ACCESS-NRI/ACCESS-MOPPeR)

The ACCESS CMIP7 consortium is actively developing the ACCESS-ESM1.6 model for submission to the CMIP7 Fasttrack initiative. As part of this effort, a spin-up run is being conducted on the NCI Gadi supercomputer, with output data progressively becoming available. However, this data cannot yet be easily evaluated using established evaluation frameworks such as [ESMValTool](https://www.esmvaltool.org/) or [ILAMB](https://www.ilamb.org/).

This notebook demonstrates how to process the variables required by the [ACCESS-ENSO-Recipe](https://github.com/ACCESS-NRI/ACCESS-ENSO-recipes), developed by ACCESS-NRI and leveraging ESMValTool, to enable robust evaluation of ACCESS-ESM1.6 spin-up runs. This workflow provides a pathway for the scientific assessment and validation of the new model output.

## Variables Required for ACCESS-NRI ENSO Recipes

To run the [ACCESS-NRI ENSO Recipes](https://github.com/ACCESS-NRI/ACCESS-ENSO-recipes) on the current ACCESS-ESM1.6 spin-up (CMIP7 FastTrack), the following variables are needed:

- **pr**: Precipitation
- **tos**: Sea surface temperature (can be interchanged with `ts`)
- **tauu**: Zonal wind stress
- **ts**: Surface air temperature (can be interchanged with `tos`)
- **zos**: Sea surface height
- **nhf**: Net heat flux, calculated as  
    `nhf = (-hfls - hfss + rlds - rlus + rsds - rsus)`

**Notes:**

- The mapping for `rlus` and `rsus` is currently unknown.
- `ts` and `tos` can be used interchangeably.

This list is based on the requirements for the ENSO diagnostics and workflow as described in the ACCESS-NRI ENSO Recipes documentation.

To run the CMORIsation we advice using an X-Large or XX-Large session on ARE.

**Note:**  
ARE (Analysis Research Environment) is the interactive computing system provided by NCI (National Computational Infrastructure). Using a larger session (X-Large or XX-Large) ensures sufficient memory and CPU resources for efficient processing of large climate datasets.

In [1]:
from access_mopper import ACCESS_ESM_CMORiser


Loaded Configuration:
Creator Name: felicity Chun
Organisation: ACCESS-NRI
Creator Email: felicity.chun@anu.edu.au
Creator URL: 0009-0007-0845-0953


In [2]:
import dask.distributed as dask

client = dask.Client(threads_per_worker = 1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 24
Total threads: 24,Total memory: 95.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40553,Workers: 24
Dashboard: /proxy/8787/status,Total threads: 24
Started: Just now,Total memory: 95.00 GiB
Comm: tcp://127.0.0.1:35855,Total threads: 1
Dashboard: /proxy/46787/status,Memory: 3.96 GiB
Nanny: tcp://127.0.0.1:45919,


We are currently using the June Spin Up dataset located at:

`/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/`

This folder contains the output files from the ACCESS-ESM1.6 spin-up experiment, which are used throughout this notebook for CMORisation and evaluation.

In [3]:
ROOT_FOLDER = "/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/"

In [4]:
OUTPUT_FOLDER = "/scratch/p66/fc6164/MOPPeR/" #"/scratch/iq82/fc6164/MOPPeR" 

In [5]:
import glob
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")

In [6]:
len(FILES), FILES[:5]


(6000,
 ['/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/output275/atmosphere/netCDF/aiihca.pa-136706_mon.nc',
  '/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/output275/atmosphere/netCDF/aiihca.pa-136709_mon.nc',
  '/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/output275/atmosphere/netCDF/aiihca.pa-136703_mon.nc',
  '/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/output275/atmosphere/netCDF/aiihca.pa-136701_mon.nc',
  '/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/output275/atmosphere/netCDF/aiihca.pa-136712_mon.nc'])

In [7]:
parent_experiment_config = {
        "parent_experiment_id": "piControl",
        "parent_activity_id": "CMIP",
        "parent_source_id": "ACCESS-ESM1-5",
        "parent_variant_label": "r1i1p1f1",
        "parent_time_units": "days since 0001-01-01 00:00:00",
        "parent_mip_era": "CMIP6",
        "branch_time_in_child": 0.0,
        "branch_time_in_parent": 54786.0,
        "branch_method": "standard"
}

### Amon.pr

In [8]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [9]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.pr",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [10]:
cmoriser.run()

In [11]:
import xarray as xr
ds = cmoriser.to_dataset()
for var, val in ds.data_vars.items():
    if var == 'pr':
        darray = val # Dask graph	6000 chunks in 12002 graph layers
        # save bnds? to apply to coords

# darray = darray.sortby('time') ## rechunk?
esm16_cube = darray.to_iris()

In [26]:
import access_mopper
print(access_mopper.__path__[0])

/home/189/fc6164/esmValTool/repos/ACCESS-MOPPeR/src/access_mopper


In [12]:
t_coord = esm16_cube.coord('time')
timels = [d for d in t_coord.units.num2date(t_coord.points)]
print(timels[0], timels[-1])
timels.sort()
timels[0], timels[-1]

1092-01-16 12:00:00 1591-12-16 12:00:00


/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/iris/common/mixin.py:195: FutureWarning: You are using legacy date precision for Iris units - max precision is seconds. In future, Iris will use microsecond precision - available since cf-units version 3.3 - which may affect core behaviour. To opt-in to the new behaviour, set `iris.FUTURE.date_microseconds = True`.
  warnings.warn(message, category=FutureWarning)


(cftime.DatetimeProlepticGregorian(1092, 1, 16, 12, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(1591, 12, 16, 12, 0, 0, 0, has_year_zero=True))

In [24]:
fp = "/scratch/p66/fc6164/MOPPeR/pr_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_136706-157204.nc"
import iris
iris.load(fp)
cube = iris.load(fp, 'pr')[0]
t_coord = cube.coord('time')
timels = [d for d in t_coord.units.num2date(t_coord.points)]
print(timels[0], timels[-1])

1367-06-16 00:00:00 1572-04-16 00:00:00


In [13]:
cmoriser.write() #write setting output folder

CMORised output written to /scratch/p66/fc6164/MOPPeR/pr_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc


In [17]:
!ls /scratch/iq82/fc6164/
# !mkdir /scratch/iq82/fc6164/MOPPeR

MOPPeR
esmvaltool_output
esmvaltool_outputs
pcmdi_metrics
pr_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_136706-157204.nc
tmp


### Omon.tos

In [4]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/ocean/ocean-2d-surface_temp-1monthly-mean*.nc")
len(FILES)

500

In [41]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Omon.tos",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    parent_info=parent_experiment_config, # <-- This is optional, can be skipped if not needed
    output_path=OUTPUT_FOLDER
    )

In [42]:
cmoriser.run()

In [43]:
cmoriser.write()

CMORised output written to /scratch/iq82/fc6164/MOPPeR/tos_Omon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_136701-157212.nc


### Amon.tauu

In [19]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [20]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.tauu",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [21]:
cmoriser.run()

In [31]:
# type(cmoriser)
# del cmoriser, #cancel(futures)
client.restart()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 24
Total threads: 24,Total memory: 95.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34089,Workers: 24
Dashboard: /proxy/8787/status,Total threads: 24
Started: 1 hour ago,Total memory: 95.00 GiB
Comm: tcp://127.0.0.1:45667,Total threads: 1
Dashboard: /proxy/41207/status,Memory: 3.96 GiB
Nanny: tcp://127.0.0.1:39933,


In [24]:
cmoriser.write()

Exception ignored in: <function CachingFileManager.__del__ at 0x153a098a2980>
Traceback (most recent call last):
  File "/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/xarray/backends/file_manager.py", line 250, in __del__
    self.close(needs_lock=False)
  File "/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/xarray/backends/file_manager.py", line 234, in close
    file.close()
  File "src/netCDF4/_netCDF4.pyx", line 2618, in netCDF4._netCDF4.Dataset.close
  File "src/netCDF4/_netCDF4.pyx", line 2585, in netCDF4._netCDF4.Dataset._close
  File "src/netCDF4/_netCDF4.pyx", line 2113, in netCDF4._netCDF4._ensure_nc_success
RuntimeError: NetCDF: Not a valid ID
2025-08-04 15:49:02,150 - distributed.worker - WARNING - Compute Failed
Key:       ('open_dataset-concatenate-62fc4ff65ad69e8674d3e4538d20f1cb', 1868, 0, 0)
Function:  getter
args:      (ImplicitToExplicitIndexingAdapter(array=CopyOnWriteArray(array=LazilyIndexedArray(array=<xarr

RuntimeError: NetCDF: Not a valid ID

### Amon.ts

In [44]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [46]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.ts",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5", #controlled variable
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [47]:
cmoriser.run()

In [48]:
cmoriser.write()

CMORised output written to /scratch/iq82/fc6164/MOPPeR/ts_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_136706-157204.nc


In [49]:
!ls /scratch/iq82/fc6164/MOPPeR

pr_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_136706-157204.nc
tauu_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_136706-157204.nc
tos_Omon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_136701-157212.nc
ts_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_136706-157204.nc
zos_Omon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_136701-157212.nc


### Omon.zos

In [35]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/ocean/ocean-2d-sea_level-1monthly-mean*.nc")
len(FILES)

500

In [36]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Omon.zos",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [37]:
cmoriser.run()

In [38]:
cmoriser.write()

CMORised output written to /scratch/iq82/fc6164/MOPPeR/zos_Omon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_136701-157212.nc


### Amon.hfss

In [8]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [9]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.hfss",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [10]:
cmoriser.run()

In [11]:
cmoriser.write()

CMORised output written to /scratch/iq82/fc6164/MOPPeR/hfss_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_136706-157204.nc


### Amon.hfls

In [12]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.hfls",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [13]:
cmoriser.run()

In [14]:
cmoriser.write()

CMORised output written to /scratch/iq82/fc6164/MOPPeR/hfls_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_136706-157204.nc


### Amon.rlds

In [15]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [16]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.rlds",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [17]:
cmoriser.run()

In [18]:
cmoriser.write()

CMORised output written to /scratch/iq82/fc6164/MOPPeR/rlds_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_136706-157204.nc


### Amon.rlus

In [19]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [20]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.rlus",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [21]:
cmoriser.run()

KeyError: 'fld_s02i20'

In [ ]:
cmoriser.write()

In [22]:
client.close()

2025-08-14 16:30:23,432 - distributed.nanny - WARNING - Worker process still alive after 3.1999986267089846 seconds, killing
2025-08-14 16:30:23,469 - distributed.nanny - WARNING - Worker process still alive after 3.1999987792968754 seconds, killing
2025-08-14 16:30:23,474 - distributed.nanny - WARNING - Worker process still alive after 3.1999989318847657 seconds, killing
2025-08-14 16:30:23,475 - distributed.nanny - WARNING - Worker process still alive after 3.1999992370605472 seconds, killing
2025-08-14 16:30:23,476 - distributed.nanny - WARNING - Worker process still alive after 3.1999986267089846 seconds, killing
2025-08-14 16:30:23,477 - distributed.nanny - WARNING - Worker process still alive after 3.1999989318847657 seconds, killing
2025-08-14 16:30:23,477 - distributed.nanny - WARNING - Worker process still alive after 3.1999989318847657 seconds, killing
2025-08-14 16:30:23,478 - distributed.nanny - WARNING - Worker process still alive after 3.1999986267089846 seconds, killing


TimeoutError: 

### Amon.rsds

In [ ]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

In [ ]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.rsds",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [ ]:
cmoriser.run()

In [ ]:
cmoriser.write()

### Amon.rsus

In [ ]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

In [ ]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.rsus",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [ ]:
cmoriser.run()

In [ ]:
cmoriser.write()